# PySESync: SE-Sync Python bindings 🐍

This notebook demonstrates the use of SE-Sync's Python bindings

In [2]:
import sys
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import collections as mc
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Line3DCollection
import matplotlib.animation as animation

%matplotlib notebook

Import the SE-Sync Python library built using pybind

In [9]:
import PySESync

Load in some data from a .g2o file

In [16]:
# filename = "../data/intel.g2o"
filename = "../data/parking-garage.g2o"


measurements, num_poses = PySESync.read_g2o_file(filename)

d = measurements[0].R.shape[0]

print("Loaded %d measurements between %d %d-dimensional poses from file %s" % (len(measurements), num_poses, d, filename))

Loaded 6275 measurements between 1661 3-dimensional poses from file ../data/parking-garage.g2o


### Run SE-Sync!

Set SE-Sync options

In [18]:
opts = PySESync.SESyncOpts()
opts.num_threads = 4
opts.verbose=True

opts.r0 = d
opts.formulation = PySESync.Formulation.Explicit    # Options are Simplified or Explicit
opts.initialization = PySESync.Initialization.Random   # Options are Chordal or Random

# Termination criteria
opts.rel_func_decrease_tol = 1e-6
opts.min_eig_num_tol = 1e-3
opts.max_time = 900

# We will record the sequence of iterates generated by the SE-Sync algorithm as it runs,
# in order to generate an animation of the solution process
opts.log_iterates = True


# Save the solution animation to a video file?
save_animation = False

Run SE-Sync!

In [19]:
result = PySESync.SESync(measurements, opts)

Constructing SE-Sync problem instance ... elapsed computation time: 0.213 seconds

========= SE-Sync ==========

ALGORITHM SETTINGS:

SE-Sync settings:
 SE-Sync problem formulation: Explicit
 Initial level of Riemannian staircase: 3
 Maximum level of Riemannian staircase: 10
 Tolerance for accepting an eigenvalue as numerically nonnegative in optimality verification: 0.001
 LOBPCG block size: 4
 LOBPCG preconditioner maximum fill factor: 3
 LOBPCG preconditioner drop tolerance: 0.001
 Maximum number of LOBPCG iterations for escape direction computation: 100
 Initialization method: random
 Logging entire sequence of Riemannian Staircase iterates
 Running SE-Sync with 4 threads

Riemannian trust-region settings:
 Stopping tolerance for norm of Riemannian gradient: 0.01
 Stopping tolerance for norm of preconditioned Riemannian gradient: 0.0001
 Stopping tolerance for relative function decrease: 1e-06
 Stopping tolerance for the norm of an accepted update step: 0.001
 Maximum number of tru

## Plot some interesting data about the solution process

Plot the sequence of gradient norms

In [22]:
# Plot sequence of Riemannian gradient norms
fig, ax = plt.subplots()
# Extract sequences of gradient and preconditioned gradient norms
grad_norms = np.concatenate(result.gradient_norms)
precon_grad_norms = np.concatenate(result.preconditioned_gradient_norms)

# Plot these sequences
plt.semilogy(grad_norms, '-b')
plt.semilogy(precon_grad_norms, '--c')
plt.legend(['Gradient', 'Preconditioned gradient'])

# Set the following flag to "True" in order to draw red vertical 
# lines indicating iterations at which the relaxation rank 
# increased in the Riemannian Staircase
show_rank_increases = True

if show_rank_increases and len(result.gradient_norms) > 1:
    steps = np.cumsum([len(l) for l in result.gradient_norms[:-1]])
    for i in steps:
        plt.axvline(x=i, ymin=0, ymax=np.log10(grad_norms.max()), color='red', linestyle='--')

plt.title("Gradient norms")
plt.xlabel("Iteration")

plt.show()

<IPython.core.display.Javascript object>

Plot the sequence of update step sizes

In [23]:
# Plot sequence of update step norms
fig, ax = plt.subplots()
# Extract sequences of gradient and preconditioned gradient norms
update_step_norms = np.concatenate(result.update_step_norms)
update_step_M_norms = np.concatenate(result.update_step_M_norms)

# Plot these sequences
plt.semilogy(update_step_norms, '-b')
plt.semilogy(update_step_M_norms, '--c')
plt.legend(['Frobenius norm', 'M norm'])

# Set the following flag to "True" in order to draw red vertical 
# lines indicating iterations at which the relaxation rank 
# increased in the Riemannian Staircase
show_rank_increases = True

if show_rank_increases and len(result.gradient_norms) > 1:
    steps = np.cumsum([len(l) for l in result.gradient_norms[:-1]])
    for i in steps:
        plt.axvline(x=i, ymin=0, ymax=np.log10(grad_norms.max()), color='red', linestyle='--')

plt.title("Update step norms")
plt.xlabel("Iteration")

plt.show()

<IPython.core.display.Javascript object>

Plot the sequence of gain ratios for the computed update steps

In [24]:
# Plot sequence of gain ratio values: this measures how well the second-order local model
# of the objective used to calculate the update steps h agrees with the *actual* objective F.
#
# This value should be close to 1 if the local model is accurately capturing the shape of the objective F over
# a region of the same size as the length of the update steps h
#
# Blue markers indicate iterations for which the gain ratio is positive (corresponding to accepted steps),
# while red markers indicate iterations for which the gain ratio is negative (corresponding to rejected steps)

fig, ax = plt.subplots()
gain_ratios = np.concatenate(result.gain_ratios)

# Plot sequence of nonnegative gain ratios with blue markers
nonnegative_rho_indicators = (gain_ratios >= 0)
nonnegative_rho_indxs = np.nonzero(nonnegative_rho_indicators)[0]
plt.plot(nonnegative_rho_indxs, gain_ratios[nonnegative_rho_indicators], \
         color='blue', marker=".", linestyle='None')

# Extract the set of iterations in which the gain ratio is less than 0;
# these correspond to iterations in which the proposed update was rejected
neg_rho_idxs = np.nonzero(gain_ratios < 0)[0]
num_neg_rhos = len(neg_rho_idxs)

# Plot these rejected steps
plt.plot(neg_rho_idxs, np.zeros(len(neg_rho_idxs)), color='red', marker='.', linestyle="None")

# Set reasonable range for viewing gain ratios
ax.set_ylim(-.1, 1.1)

# Set the following flag to "True" in order to draw red vertical 
# lines indicating iterations at which the relaxation rank 
# increased in the Riemannian Staircase
show_rank_increases = True

if show_rank_increases and len(result.function_values) > 1:
    steps = np.cumsum([len(l) for l in result.function_values[:-1]])
    for i in steps:
        plt.axvline(x=i, ymin=-2, ymax=2, color='red', linestyle='--')


plt.title("Gain ratios")
plt.xlabel("Iteration")
plt.show()

<IPython.core.display.Javascript object>

In [25]:
# Plot sequence of objective values
fig, ax = plt.subplots()
fvals = np.concatenate(result.function_values)
plt.semilogy(fvals)

# Set the following flag to "True" in order to draw red vertical 
# lines indicating iterations at which the relaxation rank 
# increased in the Riemannian Staircase
show_rank_increases = True

if show_rank_increases and len(result.function_values) > 1:
    steps = np.cumsum([len(l) for l in result.function_values[:-1]])
    for i in steps:
        plt.axvline(x=i, ymin=0, ymax=np.log10(fvals.max()), color='red', linestyle='--')


plt.title("Objective value")
plt.xlabel("Iteration")
plt.show()

<IPython.core.display.Javascript object>

### Plot estimated SLAM solution

Define some useful functions for visualizing SLAM solution

In [27]:
# Given a list of position estimates and a collection of measurements, this function constructs a list of
# pairs of the form (ti, tj), where ti and tj are the endpoints of a relative pose measurement
def construct_line_segments(t, measurements):
    lines = []
    
    for m in measurements:
        lines.append((t[:,m.i], t[:,m.j]))
    
    return lines

# Given an SESyncProblem instance, a low-rank factor Y for an estimated SDP solution X = YY', this function
# returns the rounded (projected) position estimates t
def get_rounded_position_estimates(prob, Y):
    # Get problem dimensions
    d = prob.dimension()
    n = prob.num_states()
    
    # Get relaxtion rank for the passed estimate Y
    r = Y.shape[0]
    
    # Compute rounded pose estimates
    xhat = prob.round_solution(Y)
    
    # Extract rotational and translational parts
    R0 = xhat[:, n : n + d]
    R0inv = R0.transpose()
    return np.matmul(R0inv, xhat[:, 0:n])

Plot the final SLAM estimate

In [28]:
# Extract translational states from solution xhat
xhat = result.xhat
R0inv = np.linalg.inv(xhat[:, num_poses : num_poses + d])
t = np.matmul(R0inv, xhat[:, 0:num_poses])

# Construct set of inter-pose measurements at the estimated robot positions
lines = construct_line_segments(t, measurements)

# Determine dimension of problem

# Plot the estimated solution
fig = plt.figure()

if d == 2:
    ax = plt.gca()
    ax.set_xlim(t[0, :].min(), t[0, :].max())
    ax.set_ylim(t[1, :].min(), t[1, :].max())
    ax.set_xlabel("x [m]")
    ax.set_ylabel("y [m]")
    
    lc = mc.LineCollection(lines)
    ax.add_collection(lc)
    ax.set_aspect('equal')

else:
    # d == 3
    ax = fig.add_subplot(projection='3d')
    ax.set_xlim(t[0, :].min(), t[0, :].max())
    ax.set_ylim(t[1, :].min(), t[1, :].max())
    ax.set_zlim(t[2, :].min(), t[2, :].max())
    ax.set_xlabel("x [m]")
    ax.set_ylabel("y [m]")
    ax.set_zlabel("z [m]")
    
    lc = Line3DCollection(lines)
    ax.add_collection3d(lc)
    ax.set_box_aspect((1,1,1))

plt.title("Estimated SLAM solution")
plt.show()

<IPython.core.display.Javascript object>

### Generate a video of the solution process

Compute rounded position estimates for each iterate generated during the entire optimization

In [29]:
# Construct SE-Sync problem instance
prob = PySESync.SESyncProblem(measurements, opts.formulation, opts.projection_factorization, \
                              opts.preconditioner, opts.reg_Chol_precon_max_cond)

# Record maximum and minimum x- and y- values of each estimated position; we will use these later
# to set the figure axis ranges
xmax = -float('inf')
xmin = float('inf')
ymax = -float('inf')
ymin = float('inf')
zmax = -float('inf')
zmin = float('inf')


# Construct a list of line collections, one for each estimated (rounded) solution
solns = []
for Ys in result.iterates:
    for Y in Ys:
        # Get rounded position estimates
        t = get_rounded_position_estimates(prob, Y)
        
        # Update maximum and minimum x- and y- values
        xmax = max(xmax, t[0, :].max())
        xmin = min(xmin, t[0, :].min())
        ymax = max(ymax, t[1, :].max())
        ymin = min(ymin, t[1, :].min())
        
        if d == 3:
            # Also record zmax and zmin values
            zmax = max(zmax, t[2, :].max())
            zmin = min(zmin, t[2, :].min())

        # Calculate the set of segments for this solution
        solns.append(construct_line_segments(t, measurements))

Render visualization!

In [ ]:
# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure()

if d == 2:
    # Construct set of 2D axes
    ax = plt.axes(xlim=(xmin, xmax), ylim=(ymin, ymax))
    ax.set_xlabel("x [m]")
    ax.set_ylabel("y [m]")
    ax.set_aspect('equal')

    # Initialize line collection and add it to the figure
    lc = mc.LineCollection([])
    ax.add_collection(lc)
    
else:
    # Construct set of 3D axes
    ax = fig.add_subplot(projection='3d')
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.set_zlim(zmin, zmax)
    ax.set_xlabel("x [m]")
    ax.set_ylabel("y [m]")
    ax.set_zlabel("z [m]")
    ax.set_box_aspect((1,1,1))
    
    # Initialize line collection and add it to the figure
    lc = Line3DCollection([])
    ax.add_collection3d(lc)
    


# Initialization function: plot the background of each frame
def init():
    lc.set_segments([])
    return lc

# Animation function.  This is called sequentially
def update(i):
    lc.set_segments(solns[i])
    ax.set_title("Iteration %d" % i)
    return lc

# Call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, update, init_func=init, frames=len(solns), interval=67)

if save_animation:
    # Save this animation to disk
    writer = animation.FFMpegWriter(fps = 15, codec='ffv1')
    anim.save('SESync.avi', writer=writer)